In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import least_squares
from scipy.optimize import minimize
import math
from scipy import optimize
from scipy.stats import norm, expon
from scipy import special
import numpy as np
import scipy.stats as si
import sympy as sy
from sympy.stats import Normal, cdf
import timeit

In [2]:
def ma_sd(data,n,offset):
    ma = []
    sd = []
    for i in range(0, len(data)):
        if i-n+1 - offset >= 0:
            sum_1 = []
            for j in range(0,n):
                sum_1.append(data['Adj Close'][i-j-offset])
            sd.append(np.std(sum_1))
            ma.append(sum(sum_1)/n)
        else:
            ma.append(0)
            sd.append(0)
            
            
            
    data[str(n)+'ma'] = ma
    data[str(n)+'sd'] = sd
    
    return data

In [3]:
##data cleaning 
sso = pd.read_csv('SSO.csv')
spy = pd.read_csv('SPY.csv')
spxl = pd.read_csv('SPXL.csv')
spy['Date'] = pd.to_datetime(spy['Date'])
sso['Date'] = pd.to_datetime(sso['Date'])
spxl['Date'] = pd.to_datetime(spxl['Date'])
ma_sd(spy,20,0)
ma_sd(spy,100,0)
ma_sd(spy,200,0)

,Date,Open,High,Low,Close,Adj Close,Volume,20ma,20sd,100ma,100sd,200ma,200sd
0,2000-01-03,148.250000,148.250000,143.875000,145.437500,98.564461,8164300,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2000-01-04,143.531250,144.062500,139.640625,139.750000,94.709984,8089800,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2000-01-05,139.937500,141.531250,137.250000,140.000000,94.879379,12177900,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,2000-01-06,139.625000,141.500000,137.750000,137.750000,93.354584,6227200,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,2000-01-07,140.312500,145.750000,140.062500,145.750000,98.776245,8066500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5231,2020-10-16,348.959991,350.750000,347.100006,347.290009,347.290009,89501900,337.791502,8.799709,326.391560,14.638561,309.460816,28.732318
5232,2020-10-19,348.649994,349.329987,341.040009,342.010010,342.010010,68425600,338.543503,8.479618,326.807261,14.485136,309.581623,28.817998
5233,2020-10-20,343.459991,346.880005,342.640015,343.380005,343.380005,60051900,339.197504,8.321540,327.223274,14.357171,309.703217,28.908901
5234,2020-10-21,343.329987,345.670013,342.399994,342.730011,342.730011,63575000,340.202004,7.426669,327.620590,14.230483,309.826047,28.996644


In [4]:
spy['+1sd20'] = spy['20ma'] + 1*spy['20sd']
spy['-2sd20'] = spy['20ma'] - 2*spy['20sd']
spy['+2sd20'] = spy['20ma'] + 2*spy['20sd']
spy['+1.5sd20'] = spy['20ma'] + 1.5*spy['20sd']

spy['-3sd100'] = spy['100ma'] - 3*spy['100sd']
spy['+1sd100'] = spy['100ma'] + 1*spy['100sd']

spy['-4sd200'] = spy['200ma'] - 4*spy['200sd']
spy['+1sd200'] = spy['200ma'] + 1*spy['200sd']
spy['+0.5sd100'] = spy['100ma'] +0.5*spy['100sd']
spy['+0.5sd200'] = spy['200ma'] + 0.5*spy['200sd']

In [5]:
def sell_condition_spy(date, indicator):
    if indicator == 2:
        data = spy[spy['Date'] == date]
        yes_no = (data['Adj Close'] >= data['+2sd20']).values
    elif indicator =='special':
        yes_no = False
        data = spy[spy['Date'] == date]
    
        if threshold_1 > 0:
            yes_no = (data['Adj Close'] < data['+1.5sd20']).values
    
    return yes_no

In [6]:
def spy_buy_condition(date):
    
    data = spy[spy['Date'] == date]
    yes_no = (data['Adj Close'] < data['-2sd20']).values
    
    return yes_no

def sso_buy_condition(date):
    
    data = spy[spy['Date'] == date]
    yes_no = (data['Adj Close'] < data['-3sd100']).values
    
    return yes_no

def spxl_buy_condition(date, price, n, ma):
    if ma == 100:
        sd = sd100_today
    if ma == 200:
        sd = sd200_today
    data = spy[spy['Date'] == date]
    yes_no = (data['Adj Close'].values < (price - n*sd))
    
    return yes_no

def sell_condition1(date):
    
    data = spy[spy['Date'] == date]
    yes_no = (data['Adj Close'] >= data['+2sd20']).values
        

    return yes_no

def sell_condition_special(date):
    yes_no = False
    data = spy[spy['Date'] == date]
    
    if threshold_1 > 0:
        yes_no = (data['Adj Close'] < data['+1.5sd20']).values
    
    return yes_no
    
    

def sell_condition2(date, ma, sd):
    
    
    data = spy[spy['Date'] == date]
    yes_no = (data['Adj Close'] >= data['+'+ str(sd)+'sd'+str(ma)]).values
    
    return yes_no


def sell_condition3(date, n_sd ,ma):
    
    data = spy[spy['Date'] == date]
    yes_no = (data['Adj Close'] >= data['+'+str(n_sd)+'sd'+str(ma)]).values
    
    return yes_no

def sell_condition4(date):
    
    data = spy[spy['Date'] == date]
    yes_no = (data['Adj Close'] >= data['+0.5sd200']).values
    
    return yes_no

def sell_condition5(date):
    
    data = spy[spy['Date'] == date]
    yes_no = (data['Adj Close'] >= data['+0.5sd100']).values
    
    return yes_no



In [7]:
def enter_stop_loss(date,price,n, ma_sd):
    if ma_sd == 100:
        sd = sd100_today
    if ma_sd == 200:
        sd = sd200_today
    data = spy[spy['Date'] == date]
    yes_no = (data['Adj Close'].values < (price - n*sd))
    
    return yes_no


def exit_stop_loss(date, exit_stop_loss_ma):
    data = spy[spy['Date'] == date]
    yes_no = (data['Adj Close'] >= data[str(exit_stop_loss_ma)+'ma']).values
    return yes_no

In [8]:
spy_train = spy[spy['Date'] <= '2017-12-31']
spy_test = spy[spy['Date'] > '2017-12-31']
sso_train = sso[sso['Date'] <= '2017-12-31'] 
sso_test = sso[sso['Date'] > '2017-12-31']
date_last = spy_train.index[-1]
date_first= spy[spy['Date'] == sso['Date'][0]].index.values[0]
test_date_last = spy_test.index[-1]
test_date_first = spy_train.index[-1]+1

In [9]:
def run_all(train_from, train_to , sell_spy , sell_sso_n_sd,stop_loss_n_sd,stop_loss_ma,exit_stop_loss_ma,buy_spxl_n_sd,buy_spxl_ma,sell_spxl_ma,sell_spxl_n_sd):
    
    
    

    status_list = []
    summ = 0
    status = 0
    V_0 = 10000
    total= 0
    action = 0
    spy_0 = []
    spy_t = []
    sso_0 = []
    sso_t = []
    spxl_0 = []
    spxl_t = []
    daily_vt = []
    p_l_day = []
    vt = []


    for date in spy['Date'][train_from: train_to]:
        position_balance = 0
    
        if status == 0: ### hold nothing
        
        
            if sso_buy_condition(date):
            
            
                status = 'sso'
                action = 1
            
            
            
                print(date)
                print('action: buy sso')
                SSO_buy = sso[sso['Date'] == date]['Adj Close'].values
                print('sso_buy:', SSO_buy)
            elif spy_buy_condition(date): #== True:
            
                action = 1
                status = 'spy'
            
            
                print(date)
                print('action: buy spy')
                SPY_buy = spy[spy['Date'] == date]['Adj Close'].values
                print('spy_buy:', SPY_buy)
                
                global threshold_1
                threshold_1 = 0
            
            
            else: 
            
                action = 0
                status = 0
            

            
            
            
        elif status == 'spy': ### hold spy
        
            ## special sell condition threshold 

            threshold_1 += (spy[spy['Date'] == date]['Adj Close'] >= spy[spy['Date'] == date]['+2sd20']).values
        
        
            if sso_buy_condition(date):
            
                status = 'sso'
                action = 1
                
                global sd200_today
                global sd100_today
                sd200_today = spy[spy['Date'] == date]['200sd'].values
                sd100_today = spy[spy['Date'] == date]['100sd'].values
    
                print(date)
                print('action: sell spy and buy sso')
                position_balance = 'SPY'
                SPY_sell = spy[spy['Date'] == date]['Adj Close'].values
                SSO_buy = sso[sso['Date'] == date]['Adj Close'].values
            
            
                print('spy_sell:',SPY_sell)
                print('sso_buy:' , SSO_buy)
            
            elif sell_condition_spy(date ,sell_spy):    
            
            
                status = 0
                action = 1
           
        
                print(date)
                print('action: sell spy')
                SPY_sell = spy[spy['Date'] == date]['Adj Close'].values
                position_balance = 'SPY'
                print('spy_sell:', SPY_sell)
        
            
            else:
                status = 'spy'
                action = 0
        
        
        
        
        elif status == 'sso':  ### hold sso
            if sell_condition2(date, 100,sell_sso_n_sd):
                status = 0
                action = 1
            
            
                print(date)
                print('action: sell sso')
                SSO_sell = sso[sso['Date'] == date]['Adj Close'].values
                position_balance = 'SSO'
                print('sso_sell:',SSO_sell)
            
            elif enter_stop_loss(date, SPY_sell, stop_loss_n_sd, stop_loss_ma):
                status = 'stop loss'
                action = 1
                position_balance = 'SSO'
                print(date)
            
                SSO_sell = sso[sso['Date'] == date]['Adj Close'].values
            
                print('action: sell sso stop loss')
                print('sso_sell:',SSO_sell)
            
            else: 
                status = 'sso' 
                action = 0
   

    
        elif status == 'stop loss':
        
            if exit_stop_loss(date, exit_stop_loss_ma):
                print(date)
                print('exit stop loss')
                status = 0
                action = 0
            
            elif spxl_buy_condition(date, SPY_sell, buy_spxl_n_sd , buy_spxl_ma ):
                
                status = 'spxl'
                action = 1
                SPXL_buy = spxl[spxl['Date'] == date]['Adj Close'].values
            
                print(date)
                print('buy spxl')
                print('spxl_buy:',SPXL_buy)
            
        
            else:
                status = 'stop loss'
                action = 0
    
    
    
    
    
    
        elif status == 'spxl':
            if sell_condition3(date,sell_spxl_n_sd,sell_spxl_ma ):
            
            
                status = 0
                action = 1
            
            
            
                print(date)
                print('action: sell spxl')
                position_balance = 'SPXL'
                SPXL_sell = spxl[spxl['Date'] == date]['Adj Close'].values
                print('spxl_sell:',SPXL_sell)
            
            
            
            else:
                status = 'spxl'
                action = 0
    
    
    
    
    


        #at the end of the day    ## append price
        if status == 'spy':
            spy_0.append(SPY_buy)
            spy_t.append(spy[spy['Date'] == date]['Adj Close'].values)
            stockbuy = SPY_buy
            stockprice = spy[spy['Date'] == date]['Adj Close'].values
        elif position_balance == 'SPY':
            spy_0.append(SPY_buy)
            spy_t.append(spy[spy['Date'] == date]['Adj Close'].values)
        else:
            spy_0.append(0)
            spy_t.append(0)
        
        
        if status == 'sso':
            sso_0.append(SSO_buy)
            sso_t.append(sso[sso['Date'] == date]['Adj Close'].values)
            stockbuy = SSO_buy
            stockprice = sso[sso['Date'] == date]['Adj Close'].values
        elif position_balance == 'SSO':
            sso_0.append(SSO_buy)
            sso_t.append(sso[sso['Date'] == date]['Adj Close'].values)
        else:
            sso_0.append(0)
            sso_t.append(0)
        
        if status == 'spxl':
            spxl_0.append(SPXL_buy)
            spxl_t.append(spxl[spxl['Date'] == date]['Adj Close'].values)
            stockbuy = SPXL_buy
            stockprice = spxl[spxl['Date'] == date]['Adj Close'].values
            
        elif position_balance == 'SPXL':
            spxl_0.append(SPXL_buy)
            spxl_t.append(spxl[spxl['Date'] == date]['Adj Close'].values)
        else:
            spxl_0.append(0)
            spxl_t.append(0)
       
    
    
      
    
    
    #append total(V_t)
    

        if position_balance == 'SPY':
        
            sell = SPY_sell
            buy = SPY_buy
            aa = float(sell/buy -1)
            return_rate = "{:.0%}".format(aa)
            p_l = sell/buy* 10000*0.9975 -10000
            total += p_l
            print('trade p_l:', p_l, return_rate, 'total balance:', total+10000)
        
        elif position_balance == 'SSO':
        
            sell = SSO_sell
            buy = SSO_buy
            aa = float(sell/buy -1)
            return_rate = "{:.0%}".format(aa)
            p_l = sell/buy* 10000*0.9975 -10000
            total += p_l
            print('trade p_l:', p_l, return_rate, 'total balance:', total+10000)
        
        elif position_balance == 'SPXL':
        
            sell = SPXL_sell
            buy = SPXL_buy
            aa = float(sell/buy -1)
            return_rate = "{:.0%}".format(aa)
            p_l = sell/buy* 10000*0.9975 -10000
            total += p_l
            print('trade p_l:', p_l, return_rate, 'total balance:', total+10000)

        
        
        if position_balance != 0:
            vt.append(list(total))
        else:
            vt.append(0)

    
    
    ##append
        status_list.append(status)
        
    sso_date_first = sso[sso['Date'] == spy.iloc[train_from,0]].index.values[0]
    sso_date_last = sso[sso['Date'] == spy.iloc[train_to,0]].index.values[0]
    portfolio =  pd.DataFrame(sso.iloc[sso_date_first:sso_date_last,:]['Date'])
    portfolio['holdings'] = status_list
    portfolio['spy_0'] = spy_0
    portfolio['spy_t'] = spy_t
    portfolio['sso_0'] = sso_0
    portfolio['sso_t'] = sso_t
    portfolio['spxl_0'] = spxl_0
    portfolio['spxl_t'] = spxl_t
    portfolio['daily_vt'] = 0
    portfolio['profit'] = vt
    portfolio.to_excel('portfolio.xls')
    return total

In [10]:

##spy sell
##sell_sso_n_sd: 1, 0.5
##stop_loss_n_sd: 1, 1.5, 2
##stop_loss_sd: 100 (100ma sd) , 200 (200ma sd)
##exit_stop_loss_ma: 100 200  
##buy_spxl_n_sd: 3, 3.5
##buy_spxl_ma: 100 200
##sell_spxl_ma: 100, 200
##sell_spxl_sd: 0.5, 1

## best insample result

In [11]:
run_all(date_first, date_last,'special', 1, 2, 100, 200, 3, 100, 200, 1)

2007-02-27 00:00:00
action: buy spy
spy_buy: [105.517647]
2007-04-24 00:00:00
action: sell spy
spy_sell: [112.482185]
trade p_l: [633.38528933] 7% total balance: [10633.38528933]
2007-06-07 00:00:00
action: buy spy
spy_buy: [113.226372]
2007-09-24 00:00:00
action: sell spy
spy_sell: [116.238579]
trade p_l: [240.36896214] 3% total balance: [10873.75425148]
2007-10-19 00:00:00
action: buy spy
spy_buy: [114.690666]
2007-12-11 00:00:00
action: sell spy
spy_sell: [113.341972]
trade p_l: [-142.30006564] -1% total balance: [10731.45418584]
2008-01-04 00:00:00
action: buy spy
spy_buy: [108.859154]
2008-01-17 00:00:00
action: sell spy and buy sso
spy_sell: [102.788712]
sso_buy: [15.743832]
trade p_l: [-581.24774514] -6% total balance: [10150.2064407]
2008-05-06 00:00:00
action: sell sso
sso_sell: [17.528286]
trade p_l: [1105.59696331] 11% total balance: [11255.80340401]
2008-06-06 00:00:00
action: buy spy
spy_buy: [105.51178]
2008-10-06 00:00:00
action: sell spy and buy sso
spy_sell: [81.948296

array([14234.18896974])

## out of sample result

In [13]:
run_all(test_date_first, test_date_last,'special', 1, 2, 100, 200, 3, 100, 200, 1)

2018-02-05 00:00:00
action: buy spy
spy_buy: [250.600327]
2018-05-15 00:00:00
action: sell spy
spy_sell: [258.439148]
trade p_l: [287.01970249] 3% total balance: [10287.01970249]
2018-06-27 00:00:00
action: buy spy
spy_buy: [257.92392]
2018-08-09 00:00:00
action: sell spy
spy_sell: [272.977081]
trade p_l: [557.16888521] 6% total balance: [10844.1885877]
2018-10-10 00:00:00
action: buy spy
spy_buy: [267.700623]
2018-12-20 00:00:00
action: sell spy and buy sso
spy_sell: [237.756256]
sso_buy: [44.714634]
trade p_l: [-1140.78022299] -11% total balance: [9703.40836471]
2019-02-22 00:00:00
action: sell sso
sso_sell: [56.850388]
trade p_l: [2682.26013658] 27% total balance: [12385.66850129]
2019-05-13 00:00:00
action: buy spy
spy_buy: [272.937683]
2019-07-08 00:00:00
action: sell spy
spy_sell: [289.850464]
trade p_l: [593.108092] 6% total balance: [12978.7765933]
2019-08-01 00:00:00
action: buy spy
spy_buy: [287.916931]
2019-09-16 00:00:00
action: sell spy
spy_sell: [293.112]
trade p_l: [154.

array([15790.67162391])

## parameter selections here

In [12]:
L0 = [2, 'special']
L1 = [1, 0.5]
L2=  [1, 1.5, 2]
L3=  [100, 200 ]
L4=  [100, 200 ]
L5=  [3, 3.5] 
L6=  [100,200]
L7=  [100, 200]
L8=  [0.5, 1] 

train_param = [[x,a,b,c,d,e,f,g,h] for x in L0 for a in L1 for b in L2 for c in L3 for d in L4 for e in L5 for f in L6 for g in L7 for h in L8]

In [15]:
total_return = []
count = 0
for train_input in train_param:
    pnl = train_all(date_first, 
            date_last, 
            train_input[0], 
            train_input[1], 
            train_input[2], 
            train_input[3], 
            train_input[4], 
            train_input[5], 
            train_input[6], 
            train_input[7],
            train_input[8]
           )
    print(pnl)
    total_return.append(pnl)
    count += 1
    print(count)
    
            

[12298.39106775]
1
[12131.7676084]
2
[11968.65286862]
3
[14166.31095167]
4
[2952.8647853]
5
[4719.30277882]
6
[4543.22701021]
7
[4881.07141595]
8
[8449.36565367]
9
[11129.26216285]
10
[10539.14882225]
11
[11492.33176047]
12
[4995.1298507]
13
[7130.18945181]
14
[7084.91301928]
15
[7493.25904943]
16
[12298.39106775]
17
[12131.7676084]
18
[11968.65286862]
19
[14166.31095167]
20
[2952.8647853]
21
[4719.30277882]
22
[4543.22701021]
23
[4881.07141595]
24
[8449.36565367]
25
[11129.26216285]
26
[10539.14882225]
27
[11492.33176047]
28
[4995.1298507]
29
[7130.18945181]
30
[7084.91301928]
31
[7493.25904943]
32
[12298.39106775]
33
[12131.7676084]
34
[11968.65286862]
35
[14166.31095167]
36
[2952.8647853]
37
[4719.30277882]
38
[4543.22701021]
39
[4881.07141595]
40
[8449.36565367]
41
[11129.26216285]
42
[10539.14882225]
43
[11492.33176047]
44
[4995.1298507]
45
[7130.18945181]
46
[7084.91301928]
47
[7493.25904943]
48
[12298.39106775]
49
[12131.7676084]
50
[11968.65286862]
51
[14166.31095167]
52
[2952.

In [22]:
#np.where(total_return == max(total_return))

(array([451, 467, 515, 531], dtype=int64), array([0, 0, 0, 0], dtype=int64))

In [28]:
print(train_param[451])
print(train_param[467])
print(train_param[515])
print(train_param[531])

['special', 1, 1.5, 100, 100, 3, 100, 200, 1]
['special', 1, 1.5, 100, 200, 3, 100, 200, 1]
['special', 1, 2, 100, 100, 3, 100, 200, 1]
['special', 1, 2, 100, 200, 3, 100, 200, 1]


In [70]:
run_all(date_first, date_last,'special', 1, 2, 100, 200, 3, 100, 200, 1)

2007-02-27 00:00:00
action: buy spy
spy_buy: [105.517647]
2007-04-24 00:00:00
action: sell spy
spy_sell: [112.482185]
trade p_l: [649.3753424] 7% total balance: [10649.3753424]
2007-06-07 00:00:00
action: buy spy
spy_buy: [113.226372]
2007-09-24 00:00:00
action: sell spy
spy_sell: [116.238579]
trade p_l: [255.76801322] 3% total balance: [10905.14335562]
2007-10-19 00:00:00
action: buy spy
spy_buy: [114.690666]
2007-12-11 00:00:00
action: sell spy
spy_sell: [113.341972]
trade p_l: [-127.47645672] -1% total balance: [10777.6668989]
2008-01-04 00:00:00
action: buy spy
spy_buy: [108.859154]
2008-01-17 00:00:00
action: sell spy and buy sso
spy_sell: [102.788712]
sso_buy: [15.743832]
trade p_l: [-567.08420791] -6% total balance: [10210.58269099]
2008-05-06 00:00:00
action: sell sso
sso_sell: [17.528286]
trade p_l: [1122.29710912] 11% total balance: [11332.87980011]
2008-06-06 00:00:00
action: buy spy
spy_buy: [105.51178]
2008-10-06 00:00:00
action: sell spy and buy sso
spy_sell: [81.948296]


array([14766.87196067])

In [77]:
run_all(test_date_first, test_date_last,'special', 1, 2, 100, 200, 3, 100, 200, 1)

2018-02-05 00:00:00
action: buy spy
spy_buy: [250.600327]
2018-05-15 00:00:00
action: sell spy
spy_sell: [258.439148]
trade p_l: [302.48890505] 3% total balance: [10302.48890505]
2018-06-27 00:00:00
action: buy spy
spy_buy: [257.92392]
2018-08-09 00:00:00
action: sell spy
spy_sell: [272.977081]
trade p_l: [573.04432714] 6% total balance: [10875.53323219]
2018-10-10 00:00:00
action: buy spy
spy_buy: [267.700623]
2018-12-20 00:00:00
action: sell spy and buy sso
spy_sell: [237.756256]
sso_buy: [44.714634]
trade p_l: [-1127.45808799] -11% total balance: [9748.07514421]
2019-02-22 00:00:00
action: sell sso
sso_sell: [56.850388]
trade p_l: [2701.33120446] 27% total balance: [12449.40634866]
2019-05-13 00:00:00
action: buy spy
spy_buy: [272.937683]
2019-07-08 00:00:00
action: sell spy
spy_sell: [289.850464]
trade p_l: [609.03757786] 6% total balance: [13058.44392652]
2019-08-01 00:00:00
action: buy spy
spy_buy: [287.916931]
2019-09-16 00:00:00
action: sell spy
spy_sell: [293.112]
trade p_l: [

array([15964.79293462])

In [13]:
##train
def train_all(train_from, train_to , sell_spy , sell_sso_n_sd,stop_loss_n_sd,stop_loss_ma,exit_stop_loss_ma,buy_spxl_n_sd,buy_spxl_ma,sell_spxl_ma,sell_spxl_n_sd):
    
    
    

    status_list = []
    summ = 0
    status = 0
    V_0 = 10000
    total= 0
    action = 0
    spy_0 = []
    spy_t = []
    sso_0 = []
    sso_t = []
    spxl_0 = []
    spxl_t = []
    daily_vt = []
    p_l_day = []
    vt = []


    for date in spy['Date'][train_from: train_to]:
        position_balance = 0
    
        if status == 0: ### hold nothing
        
        
            if spy_buy_condition(date): #== True:
            
                action = 1
                status = 'spy'
            
            
                ##print(date)
                ##print('action: buy spy')
                SPY_buy = spy[spy['Date'] == date]['Adj Close'].values
                ##print('spy_buy:', SPY_buy)
                
                global threshold_1
                threshold_1 = 0
            
            elif sso_buy_condition(date):
            
            
                status = 'sso'
                action = 1
            
            
            
                #print(date)
                #print('action: buy sso')
                SSO_buy = sso[sso['Date'] == date]['Adj Close'].values
                #print('sso_buy:', SSO_buy)
            
            else: 
            
                action = 0
                status = 0
            

            
            
            
        elif status == 'spy': ### hold spy
        
            ## special sell condition threshold 

            threshold_1 += (spy[spy['Date'] == date]['Adj Close'] >= spy[spy['Date'] == date]['+2sd20']).values
        
        
            if sso_buy_condition(date):
            
                status = 'sso'
                action = 1
                
                global sd200_today
                global sd100_today
                sd200_today = spy[spy['Date'] == date]['200sd'].values
                sd100_today = spy[spy['Date'] == date]['100sd'].values
    
                #print(date)
                #print('action: sell spy and buy sso')
                position_balance = 'SPY'
                SPY_sell = spy[spy['Date'] == date]['Adj Close'].values
                SSO_buy = sso[sso['Date'] == date]['Adj Close'].values
            
            
                #print('spy_sell:',SPY_sell)
                #print('sso_buy:' , SSO_buy)
            
            elif sell_condition_spy(date ,sell_spy):    
            
            
                status = 0
                action = 1
           
        
                #print(date)
                #print('action: sell spy')
                SPY_sell = spy[spy['Date'] == date]['Adj Close'].values
                position_balance = 'SPY'
                #print('spy_sell:', SPY_sell)
        
            
            else:
                status = 'spy'
                action = 0
        
        
        
        
        elif status == 'sso':  ### hold sso
            if sell_condition2(date, 100,sell_sso_n_sd):
                status = 0
                action = 1
            
            
                #print(date)
                #print('action: sell sso')
                SSO_sell = sso[sso['Date'] == date]['Adj Close'].values
                position_balance = 'SSO'
                #print('sso_sell:',SSO_sell)
            
            elif enter_stop_loss(date, SPY_sell, stop_loss_n_sd, stop_loss_ma):
                status = 'stop loss'
                action = 1
                position_balance = 'SSO'
                #print(date)
            
                SSO_sell = sso[sso['Date'] == date]['Adj Close'].values
            
                #print('action: sell sso stop loss')
                #print('sso_sell:',SSO_sell)
            
            else: 
                status = 'sso' 
                action = 0
   

    
        elif status == 'stop loss':
        
            if exit_stop_loss(date, exit_stop_loss_ma):
                #print(date)
                #print('exit stop loss')
                status = 0
                action = 0
            
            elif spxl_buy_condition(date, SPY_sell, buy_spxl_n_sd , buy_spxl_ma ):
                
                status = 'spxl'
                action = 1
                SPXL_buy = spxl[spxl['Date'] == date]['Adj Close'].values
            
                #print(date)
                #print('buy spxl')
                #print('spxl_buy:',SPXL_buy)
            
        
            else:
                status = 'stop loss'
                action = 0
    
    
    
    
    
    
        elif status == 'spxl':
            if sell_condition3(date,sell_spxl_n_sd,sell_spxl_ma ):
            
            
                status = 0
                action = 1
            
            
            
                #print(date)
                #print('action: sell spxl')
                position_balance = 'SPXL'
                SPXL_sell = spxl[spxl['Date'] == date]['Adj Close'].values
                #print('spxl_sell:',SPXL_sell)
            
            
            
            else:
                status = 'spxl'
                action = 0
    

    

        if position_balance == 'SPY':
        
            sell = SPY_sell
            buy = SPY_buy
            aa = float(sell/buy -1)
            return_rate = "{:.0%}".format(aa)
            p_l = sell/buy* 10000 -10000
            total += p_l
            #print('trade p_l:', p_l, return_rate, 'total balance:', total+10000)
        
        elif position_balance == 'SSO':
        
            sell = SSO_sell
            buy = SSO_buy
            aa = float(sell/buy -1)
            return_rate = "{:.0%}".format(aa)
            p_l = sell/buy* 10000 -10000
            total += p_l
            #print('trade p_l:', p_l, return_rate, 'total balance:', total+10000)
        
        elif position_balance == 'SPXL':
        
            sell = SPXL_sell
            buy = SPXL_buy
            aa = float(sell/buy -1)
            return_rate = "{:.0%}".format(aa)
            p_l = sell/buy* 10000 -10000
            total += p_l
            #print('trade p_l:', p_l, return_rate, 'total balance:', total+10000)

        
        
  
    return total